# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [4]:
# number of callbacks for white-sounding names
sum(data[data.race=='w'].call)

235.0

In [5]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [6]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [7]:
data.shape

(4870, 65)

Q1: What test is appropriate for this problem? __The appropriate test to use would be a hypothesis test.__


Does CLT apply? __Since we are dealing with two proportion comparisons, the Central Limit Theorem would be applied if it fulfills following critieris:__

(1) Are the samples independent? __Yes. The samples are independent.__

(2) Check for Sample size/skew. np >= 10? and n(1-p) >= 10? __Yes.__

Since we don't have the actual population proportion, we'll have to compute the sample pool proportion:
Ppool = TotalSuccess / TotalSampleSize = (#Success1 + #Success2) / (Size1 + Size2) = (#WhiteCallBack + #BlackCallBack) / (TotalSampleSize)

Question 2:
What are the null and alternate hypotheses?

__Null Hypothesis: (H0: pw = pb) The null hypothesis would be that there is no racial discrimination between black/white races on rates of callback for resumes.__

__Alternate Hypothesis: (H1: pw != pb) The alternate hypothesis would be that there is some racial discrimination between black/white races on rates of callback for resumes.__

Question 3:

__Compute margin of error, confidence interval, and p-value__

__Margin of Error:__ The Margin of Error is about .0077

__Confidence Interval:__ The Confidence Interval at 95% Confidence Level is (-0.0398297482456 , -0.0242359601733)

__P-Value:__ The P-Value is 4E-5

In [10]:
data['race'].value_counts()

w    2435
b    2435
Name: race, dtype: int64

In [11]:
data['call'].value_counts()

0.0    4478
1.0     392
Name: call, dtype: int64

In [12]:
# number of callbacks and no callbacks for white-sounding names
data[data.race=='w'].call.value_counts()

0.0    2200
1.0     235
Name: call, dtype: int64

In [13]:
# number of callbacks and no callbacks for black-sounding names
data[data.race=='b'].call.value_counts()

0.0    2278
1.0     157
Name: call, dtype: int64

In [14]:
# Calculate the # of callbacks for each race
white_callback = sum(data[data.race=='w'].call)
black_callback = sum(data[data.race=='b'].call)

# Calculate the sample size for each race
white_sample_size = len(data[data.race=='w'])
black_sample_size = len(data[data.race=='b'])

# Calculate the sample pool proportion
sample_pool_proportion = (white_callback + black_callback) / (white_sample_size + black_sample_size)
sample_pool_proportion

0.08049281314168377

In [15]:
# Check both cases (np>=10 and n(1-p)>=10) for each race
whiteCLT1 = white_sample_size*sample_pool_proportion >= 10
whiteCLT2 = white_sample_size*(1-sample_pool_proportion) >= 10

blackCLT1 = black_sample_size*sample_pool_proportion >= 10
blackCLT2 = black_sample_size*(1-sample_pool_proportion) >= 10

# Print the case statement to see if it fullfills the second criteria of CLT
print (whiteCLT1,whiteCLT2, blackCLT1,blackCLT2)

True True True True


In [16]:
# Calculate the total size of the sample population
n = black_sample_size + white_sample_size

# Store the sample proportion as p
p = sample_pool_proportion

# Let our Z-Score be 1.96 at 95% Confidence Level
z = 1.96

In [17]:
import math

# Calculate the standard error / margin of error
SE = math.sqrt(p*(1-p)*((1/black_sample_size) + (1/white_sample_size)))
SE

0.007796894036170457

In [18]:
# Calculate the Confidence Interval
white_sample_proportion = white_callback / white_sample_size

black_sample_proportion = black_callback / black_sample_size

estimate_point = black_sample_proportion - white_sample_proportion 

lower_bound = estimate_point - SE 
upper_bound = estimate_point + SE

print(lower_bound, upper_bound)

-0.03982974824561604 -0.024235960173275128


In [19]:
# Calculate Z-Score and P-value

#Z-Score
z_score = estimate_point / SE
z_score

#P-Value = 4E-05

-4.108412152434346

Question 4:

__Write a story describing the statistical significance in the context or the original problem.__

After the hypothesis testing, one can support the alternative hypothesis of having some racial discrimination between black/white races on rates of callback for resumes because of the P-Value Score <0.05. 

__Conclusion__ Evidence of Racial Discrimination 

Question 5:

__Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?__

The data set have total 65 variables, only by analysing the data using only two varibales we can not firmly say that race/name can be the most important factor in callback success. We need to do more analysis to reach to a final conclusion. It could be just a correlation that race/name had an impact with rates of call back. 
